# Imports

In [1]:
import pandas as pd; import seaborn as sb; import warnings; import scipy; import re; 
import os; from analysis_helpers import *; import itertools; from scipy import stats
import random; import pandas as pd; import numpy as np; from sklearn import datasets, linear_model; 
from sklearn.linear_model import LinearRegression; import statsmodels.api as sm
from scipy import stats; from itertools import groupby; from operator import itemgetter;
import csv; import time
warnings.filterwarnings('ignore')
%matplotlib inline

# Import behavioral data 

Here, we ran a single presentation and memory run and recorded and time stamped the filename of each presented stimulus, its location, millisecond timings on window flips, etc, to verify the precision and accuracy of the data in our behavioral data files

In [2]:
with open('/Users/kirstenziman/Desktop/FINAL_EYE_2019_Sep_05/test_data/sustained_with_times_and_images_2.log') as file:
    lines = file.read().splitlines()
    lines = [[lines[x]] for x in range(0, len(lines))]
    lines.insert(0,["DATA"])

    with open('/Users/kirstenziman/Desktop/FINAL_EYE_2019_Sep_05/test_data/sustained_with_times_and_images_2.csv', 'w+') as csvfile:
        w = csv.writer(csvfile)
        w.writerows(lines)

# Log file data

In [3]:
log_file = pd.read_csv('/Users/kirstenziman/Desktop/FINAL_EYE_2019_Sep_05/test_data/sustained_with_times_and_images_2.csv')
log_file = log_file.reset_index()
log_file['TIME'],log_file['WARNING'],log_file['MESSAGE'] = log_file['DATA'].str.split('\t', 4).str
log_file = log_file.fillna('NAN VALUE')

In [4]:
log_file.head()

,index,DATA,TIME,WARNING,MESSAGE
0,0,4.7915 \tWARNING \twarning,4.7915,WARNING,warning
1,1,4.7916 \tEXP \texperiment,4.7916,EXP,experiment
2,2,4.7916 \tDATA \tdata,4.7916,DATA,data
3,3,4.7916 \tINFO \tinfo,4.7916,INFO,info
4,4,4.7916 \tINFO \tcurrent time: 1567699848.41,4.7916,INFO,current time: 1567699848.41


In [5]:
log_file['TIME'] = log_file['TIME'].str.replace(' ','')
log_file['TIME'] = log_file['TIME'].astype(float, errors='ignore')

In [6]:
p = log_file[(log_file['DATA'].str.contains('COMPOSITE'))
    & ((log_file['DATA'].str.contains('ON')))].melt(id_vars=['MESSAGE'], value_vars=['TIME'])

In [7]:
p['utc'] = [x[-13:] for x in list(p['value'])]

In [8]:
p.head()

,MESSAGE,variable,value,utc
0,NAN VALUE,TIME,ameCUEDCOMPOSITE../../stim/practice_composite/...,1567699863.21
1,NAN VALUE,TIME,ameUNCUEDCOMPOSITE../../stim/practice_composit...,1567699863.23
2,NAN VALUE,TIME,ameCUEDCOMPOSITE../../stim/practice_composite/...,1567699867.37
3,NAN VALUE,TIME,ameUNCUEDCOMPOSITE../../stim/practice_composit...,1567699867.38
4,NAN VALUE,TIME,ameCUEDCOMPOSITE../../stim/practice_composite/...,1567699871.44


In [9]:
uncued = p[ (p['value'].str.contains('UNCUED')) & (p['value'].str.contains('ON')) & (~p['value'].str.contains('practice'))]
cued   = p[(~p['value'].str.contains('UNCUED')) & (p['value'].str.contains('ON')) & (~p['value'].str.contains('practice'))]
cued = cued.rename(columns={"value": "cued_onset"})
uncued = uncued.rename(columns={"value": "uncued_onset"})

cued   = cued.reset_index(drop=True)
uncued = uncued.reset_index(drop=True)

In [10]:
uncued['utc'][0] = uncued['utc'][0][1:]
uncued['utc'][1] = uncued['utc'][1][1:]

In [11]:
uncued

,MESSAGE,variable,uncued_onset,utc
0,NAN VALUE,TIME,ameUNCUEDCOMPOSITE../../stim/composite/0093396...,1567699909.8
1,NAN VALUE,TIME,ameUNCUEDCOMPOSITE../../stim/composite/0055994...,1567699914.5
2,NAN VALUE,TIME,ameUNCUEDCOMPOSITE../../stim/composite/0077494...,1567699919.52
3,NAN VALUE,TIME,ameUNCUEDCOMPOSITE../../stim/composite/0088496...,1567699924.34
4,NAN VALUE,TIME,ameUNCUEDCOMPOSITE../../stim/composite/0083594...,1567699929.02
5,NAN VALUE,TIME,ameUNCUEDCOMPOSITE../../stim/composite/0015493...,1567699933.64
6,NAN VALUE,TIME,ameUNCUEDCOMPOSITE../../stim/composite/0031394...,1567699938.39
7,NAN VALUE,TIME,ameUNCUEDCOMPOSITE../../stim/composite/0056894...,1567699943.24
8,NAN VALUE,TIME,ameUNCUEDCOMPOSITE../../stim/composite/0009093...,1567699947.94
9,NAN VALUE,TIME,ameUNCUEDCOMPOSITE../../stim/composite/0070596...,1567699952.55


In [12]:
cued['utc'][2] = cued['utc'][2][1:]

In [13]:
cued

,MESSAGE,variable,cued_onset,utc
0,NAN VALUE,TIME,ameCUEDCOMPOSITE../../stim/composite/001469606...,1567699909.78
1,NAN VALUE,TIME,ameCUEDCOMPOSITE../../stim/composite/009799606...,1567699914.48
2,NAN VALUE,TIME,ameCUEDCOMPOSITE../../stim/composite/008829605...,1567699919.5
3,NAN VALUE,TIME,ameCUEDCOMPOSITE../../stim/composite/010099606...,1567699924.31
4,NAN VALUE,TIME,ameCUEDCOMPOSITE../../stim/composite/007729412...,1567699929.01
5,NAN VALUE,TIME,ameCUEDCOMPOSITE../../stim/composite/009479606...,1567699933.62
6,NAN VALUE,TIME,ameCUEDCOMPOSITE../../stim/composite/007709412...,1567699938.38
7,NAN VALUE,TIME,ameCUEDCOMPOSITE../../stim/composite/009429606...,1567699943.23
8,NAN VALUE,TIME,ameCUEDCOMPOSITE../../stim/composite/009789606...,1567699947.92
9,NAN VALUE,TIME,ameCUEDCOMPOSITE../../stim/composite/007779412...,1567699952.54


In [14]:
uncued['utc'] = uncued['utc'].astype(float)
cued['utc']   = cued['utc'].astype(float)

In [15]:
uncued['uncued_onset'][0]

'ameUNCUEDCOMPOSITE../../stim/composite/00933960627fa_sunapwjkcxdaeqehfql.jpgON\\utc1567699909.8'

# average difference in onset times between Cued and Uncued Composite in a single test block (~1 frame)

In [16]:
x = list(uncued['utc']-cued['utc'])

In [17]:
print('Average number of seconds cued composite shows before uncued composite in a test run '+str(np.mean(x)))

Average number of seconds cued composite shows before uncued composite in a test run 0.01700000762939453


# Check A: order of stimuli, log versus csv files

## verify stimulus orders same as in behavioral csv

### load behavioral csv

In [18]:
sub_list = []
data_dir = '/Users/kirstenziman/Desktop/FINAL_EYE_2019_Sep_05'

for sub_dir in os.listdir(data_dir):
    sub_list.append(add_level(sum_pd(data_dir + '/' + sub_dir)))
    print(sub_dir)
    
exp_raw = pd.concat(sub_list)
#exp1_raw.to_csv('../parsed_data/behavioral_data_sustained.csv')

test_data


### Cued composite images are in the same order in log file and csv

In [19]:
# cued composite stimuli from log file
[x[36:71] for x in list(cued['cued_onset'])]

['/00146960620fb_sunaatpeezlljqaxlhj.',
 '/00979960627fa_sunbdycacjfaujxitbw.',
 '/00882960530fa_sunaknlirqjqiwbbeeg.',
 '/01009960627fa_sunbxzweleltykebgan.',
 '/00772941201fa_sunafpfpiiujzoesnqg.',
 '/00947960627fa_sunasmmcxdddtmtazwx.',
 '/00770941201fa_sunaflojskatpeqyzqp.',
 '/00942960627fa_sunarfdklgdnjgeqfjg.',
 '/00978960627fa_sunbddbivbptlrlauhy.',
 '/00777941205fa_sunagcinxkccdtrjmlv.']

In [20]:
# cued composite stimuli from csv
exp_raw[exp_raw['Trial Type']=='Presentation']['Cued Composite']

0    00146960620fb_sunaatpeezlljqaxlhj.jpg
1    00979960627fa_sunbdycacjfaujxitbw.jpg
2    00882960530fa_sunaknlirqjqiwbbeeg.jpg
3    01009960627fa_sunbxzweleltykebgan.jpg
4    00772941201fa_sunafpfpiiujzoesnqg.jpg
5    00947960627fa_sunasmmcxdddtmtazwx.jpg
6    00770941201fa_sunaflojskatpeqyzqp.jpg
7    00942960627fa_sunarfdklgdnjgeqfjg.jpg
8    00978960627fa_sunbddbivbptlrlauhy.jpg
9    00777941205fa_sunagcinxkccdtrjmlv.jpg
Name: Cued Composite, dtype: object

### Uncued composite images are in the same order in log file and csv

In [21]:
# uncued composite from log
[x[36:71] for x in list(uncued['uncued_onset'])]

['te/00933960627fa_sunapwjkcxdaeqehfq',
 'te/00559940519fa_sunacjkmnpawfzbyva',
 'te/00774941201fa_sunafvtpfczkzopbgu',
 'te/00884960530fa_sunakodkcslgqhqlgw',
 'te/00835940307fa_sunaijsqpvodplbwxk',
 'te/00154931230fa_sunaauueuhhvklldji',
 'te/00313940422fa_sunabwjvmoaddlyxfj',
 'te/00568940928fa_sunacsdhbbsybcyxne',
 'te/00090931230fa_sunaahedjlmtqekgyv',
 'te/00705960530fa_sunaduslxfzlbxllln']

In [22]:
# uncued composite from csv
exp_raw[exp_raw['Trial Type']=='Presentation']['Uncued Composite']

0    00933960627fa_sunapwjkcxdaeqehfql.jpg
1    00559940519fa_sunacjkmnpawfzbyvae.jpg
2    00774941201fa_sunafvtpfczkzopbgue.jpg
3    00884960530fa_sunakodkcslgqhqlgwl.jpg
4    00835940307fa_sunaijsqpvodplbwxkf.jpg
5    00154931230fa_sunaauueuhhvklldjik.jpg
6    00313940422fa_sunabwjvmoaddlyxfjr.jpg
7    00568940928fa_sunacsdhbbsybcyxnel.jpg
8    00090931230fa_sunaahedjlmtqekgyvj.jpg
9    00705960530fa_sunaduslxfzlbxlllnz.jpg
Name: Uncued Composite, dtype: object

### Onset times differ by fractions of a second

In [23]:
d = {}

# aggregate onset times (two onsets in log file, one for cued composite and one for uncued; csv has only one onset time assigned to both)
d['csv_cued_onset'] = list(exp_raw[exp_raw['Trial Type']=='Presentation']['Stimulus Onset'])
d['log_cued_onset'] = list(cued['utc'])
d['log_uncued_onset'] = list(uncued['utc'])

# aggregate stim end times
p = log_file[(log_file['DATA'].str.contains('OFF')) & ((log_file['DATA'].str.contains('COMPOSITE'))) & ((~log_file['DATA'].str.contains('practice')))]
p['utc'] = [x[-13:] for x in list(p['DATA'])]
d['log_composite_off'] = [float(x) for x in p['utc']]
d['csv_composite_off'] = list(exp_raw[exp_raw['Trial Type']=='Presentation']['Stimulus End'])

### check time diffs

In [24]:
d = pd.DataFrame(d)
d

,csv_cued_onset,log_cued_onset,log_uncued_onset,log_composite_off,csv_composite_off
0,1.567700e+09,1.567700e+09,1.567700e+09,1.567700e+09,1.567700e+09
1,1.567700e+09,1.567700e+09,1.567700e+09,1.567700e+09,1.567700e+09
2,1.567700e+09,1.567700e+09,1.567700e+09,1.567700e+09,1.567700e+09
3,1.567700e+09,1.567700e+09,1.567700e+09,1.567700e+09,1.567700e+09
4,1.567700e+09,1.567700e+09,1.567700e+09,1.567700e+09,1.567700e+09
5,1.567700e+09,1.567700e+09,1.567700e+09,1.567700e+09,1.567700e+09
6,1.567700e+09,1.567700e+09,1.567700e+09,1.567700e+09,1.567700e+09
7,1.567700e+09,1.567700e+09,1.567700e+09,1.567700e+09,1.567700e+09
8,1.567700e+09,1.567700e+09,1.567700e+09,1.567700e+09,1.567700e+09
9,1.567700e+09,1.567700e+09,1.567700e+09,1.567700e+09,1.567700e+09


In [25]:
d['cue_on_diff'] = d['csv_cued_onset']-d['log_cued_onset']
d['uncue_on_diff'] = d['csv_cued_onset']-d['log_uncued_onset']

print('mean onset difference between csv and log file, cued composites: '+str(d['cue_on_diff'].mean()))
print('^^ this is approximately two frames')
print()

print('mean onset difference between csv andn log file, uncued composites: '+str(d['uncue_on_diff'].mean()))
print('^^ this is approximately one frame')

mean onset difference between csv and log file, cued composites: 0.033507490158081056
^^ this is approximately two frames

mean onset difference between csv andn log file, uncued composites: 0.016507482528686522
^^ this is approximately one frame


In [26]:
d['composite_off_diff'] = d['csv_composite_off']-d['log_composite_off']

print('mean end time difference between csv and log file, for both composites: '+str(d['composite_off_diff'].mean()))
print('^^ this is approximately one frame')

mean end time difference between csv and log file, for both composites: -0.01654200553894043
^^ this is approximately one frame


# Check Memory Images

Pull ordered memory images from log file and csv

In [27]:
image_dict = {}
image_dict['mem_from_log'] = [x[-45:-20] for x in list(log_file[(log_file['DATA'].str.contains('MEMORY')) & (log_file['DATA'].str.contains('ON')) & (~log_file['DATA'].str.contains('practice'))]['DATA'])]
image_dict['mem_from_csv'] = list(exp_raw[exp_raw['Trial Type']=='Memory']['Memory Image'])

In [28]:
pd.DataFrame(image_dict)

,mem_from_log,mem_from_csv
0,single/00762941201fa.jpg,00762941201fa.jpg
1,/single/00194940128fb.jpg,00194940128fb.jpg
2,single/00313940422fa.jpg,00313940422fa.jpg
3,single/01009960627fa.jpg,01009960627fa.jpg
4,/sunaauueuhhvklldjik.jpg,sunaauueuhhvklldjik.jpg
5,single/00882960530fa.jpg,00882960530fa.jpg
6,single/00705960530fa.jpg,00705960530fa.jpg
7,single/00144931230fb.jpg,00144931230fb.jpg
8,/sunafgzgbisbopcyslr.jpg,sunafgzgbisbopcyslr.jpg
9,e/sunawotxrevnqvamsfp.jpg,sunawotxrevnqvamsfp.jpg


Onset times from log

In [29]:
d2={}

In [30]:
onset_list = [x[-13:] for x in list(log_file[(log_file['DATA'].str.contains('MEMORY')) & (log_file['DATA'].str.contains('ON')) & (~log_file['DATA'].str.contains('practice'))]['DATA'])]
d2['memory_onset_log'] = [float(x.replace("c", "")) for x in onset_list]
#onset_list

Onset times from csv

In [31]:
d2['memory_onset_csv'] = list(exp_raw[exp_raw['Trial Type']=='Memory']['Stimulus Onset'])

Aggregate end times from log and csv

In [32]:
d2['memory_off_log']=[x[-13:] for x in list(log_file[(log_file['DATA'].str.contains('MEMORY')) & (log_file['DATA'].str.contains('OFF')) & (~log_file['DATA'].str.contains('practice'))]['DATA'])]
d2['memory_off_csv'] = list(exp_raw[exp_raw['Trial Type']=='Memory']['Stimulus End'])

In [33]:
d2 = pd.DataFrame(d2)
d2.head()

,memory_onset_log,memory_onset_csv,memory_off_log,memory_off_csv
0,1.567700e+09,1.567700e+09,1567699961.68,1.567700e+09
1,1.567700e+09,1.567700e+09,1567699965.06,1.567700e+09
2,1.567700e+09,1.567700e+09,1567699968.44,1.567700e+09
3,1.567700e+09,1.567700e+09,1567699971.82,1.567700e+09
4,1.567700e+09,1.567700e+09,1567699975.22,1.567700e+09


In [34]:
d2['mem_onset_diff'] = d2['memory_onset_csv'] - d2['memory_onset_log']

In [35]:
print('Average difference in memory image onset time between csv and log: '+str(d2['mem_onset_diff'].mean()))

Average difference in memory image onset time between csv and log: 0.029798215627670287


In [36]:
d2['memory_off_log'] = pd.to_numeric(d2['memory_off_log'])
d2['mem_off_diff'] = d2['memory_off_csv'] - d2['memory_off_log']

In [37]:
print('Average difference in end time between csv and log: '+str(d2['mem_off_diff'].mean()))

Average difference in end time between csv and log: -0.2511367857456207


this longer lag in end time is likely due to the amount of time taken to extract the rating from the rating scale. Can time this, to check, and also move the rating scale data pulling code to after the stimulus has been pulled from the screen

# Check Responses and Reaction Times

Here we see that the ratings and reaction times are the same across the log files and csv. Note that the log file does not record a spearate line for the rating scale history when no response is recorded. So, every Nan in the memory data from the csv corresponds with a skipped row in the list below.

In [38]:
# Note, we do not show the first four rating scales because these are from the practice/instruction round
# that is, we ignore the first 8 list items (RT and response info (2) for each practice image (4) --> 2*4 = 8)
[x for x in list(log_file[((log_file['DATA'].str.contains('history')) | (log_file['DATA'].str.contains('RT')))]['DATA'])][8:]

['111.4919 \tDATA \tRatingScale unnamed RatingScale: rating RT=0.536',
 '111.4919 \tDATA \tRatingScale unnamed RatingScale: history=[(None, 0.0), (1, 0.536)]',
 '114.7773 \tDATA \tRatingScale unnamed RatingScale: rating RT=0.370',
 '114.7774 \tDATA \tRatingScale unnamed RatingScale: history=[(None, 0.0), (3, 0.37)]',
 '118.1266 \tDATA \tRatingScale unnamed RatingScale: rating RT=0.335',
 '118.1266 \tDATA \tRatingScale unnamed RatingScale: history=[(None, 0.0), (2, 0.335)]',
 '121.4945 \tDATA \tRatingScale unnamed RatingScale: rating RT=0.335',
 '121.4945 \tDATA \tRatingScale unnamed RatingScale: history=[(None, 0.0), (3, 0.335)]',
 '124.8801 \tDATA \tRatingScale unnamed RatingScale: rating RT=0.336',
 '124.8801 \tDATA \tRatingScale unnamed RatingScale: history=[(None, 0.0), (4, 0.336)]',
 '128.1800 \tDATA \tRatingScale unnamed RatingScale: rating RT=0.236',
 '128.1800 \tDATA \tRatingScale unnamed RatingScale: history=[(None, 0.0), (1, 0.236)]',
 '131.5490 \tDATA \tRatingScale unnamed R

In [39]:
### show ratings and reaction times from csv file

In [40]:
exp_raw[exp_raw['Trial Type']=='Memory'][['Familiarity Rating','Familiarity Reaction Time (s)','Memory Image']]

,Familiarity Rating,Familiarity Reaction Time (s),Memory Image
10,1.0,0.536,00762941201fa.jpg
11,3.0,0.370,00194940128fb.jpg
12,2.0,0.335,00313940422fa.jpg
13,3.0,0.335,01009960627fa.jpg
14,4.0,0.336,sunaauueuhhvklldjik.jpg
15,1.0,0.236,00882960530fa.jpg
16,2.0,0.221,00705960530fa.jpg
17,3.0,0.253,00144931230fb.jpg
18,4.0,0.202,sunafgzgbisbopcyslr.jpg
19,4.0,0.386,sunawotxrevnqvamsfp.jpg


# Check attention probe RT's

In [41]:
log_file[((log_file['DATA'].str.contains('ATTENTION PROBE')) & (log_file['DATA'].str.contains('ON'))) | (log_file['DATA'].str.contains('Keypress'))].iloc[30:70][['DATA','TIME']]

,DATA,TIME
243,43.3660 \tEXP \tATTENTION PROBE: autoDraw = False,43.3660
244,43.3660 \tEXP \tWIN FLIP : STIMULI OFF ['ATTEN...,43.3660
248,45.2265 \tDATA \tKeypress: space,45.2265
317,46.9371 \tDATA \tKeypress: 1,46.9371
577,50.1224 \tDATA \tKeypress: 4,50.1224
827,53.1890 \tDATA \tKeypress: 3,53.1890
1073,56.2242 \tDATA \tKeypress: 4,56.2242
1277,58.7303 \tDATA \tKeypress: space,58.7303
1296,61.3694 \tEXP \tCreated ATTENTION PROBE = Text...,61.3694
1299,61.3892 \tEXP \tATTENTION PROBE: pos = array([...,61.3892


In [42]:
a = log_file[(log_file['DATA'].str.contains('ATTENTION PROBE ON'))]['DATA'].str.split('\t', 4).str

In [43]:
probe_indices = list(log_file[(log_file['DATA'].str.contains('ATTENTION PROBE ON'))].index)

because these lines are long, they got split into two lines when converting the log file into csv format.
if we want to look at the psychopy clock times, we have to retrieve them from the line before the attention probe line. so, we get the indices of the lines where the second half of the attention probe info displays, and subtract each by 1

In [44]:
probe_time_indices = [x-1 for x in probe_indices]
key_press_indices = [x+1 for x in probe_indices]

First we assume that the first button they press after seeing the probe is a '1' or a '3'. Then, for any instances where that isnt the case, we loop through their next button presses until we find the instance where they pressed a 1 or a 3, allowing the experiment to continue

In [45]:
for idx,x in enumerate(key_press_indices):
    while 'Keypress: 1' not in log_file.loc[x]['DATA'] and 'Keypress: 3' not in log_file.loc[x]['DATA']:
        x+=1
    key_press_indices[idx]=x

In [46]:
probe_time_indices

[198, 219, 240, 1310, 1331, 1352, 1373, 1394, 1415, 1436, 1457, 1478, 1500]

In [47]:
full_index = probe_time_indices+key_press_indices+probe_indices
full_index.sort()

In [48]:
attn_rt = {}
attn_rt['probe_start'] = list(log_file.loc[probe_time_indices]['TIME'][3:])
attn_rt['key press'] = list(log_file.loc[key_press_indices]['TIME'][3:])
attn_df = pd.DataFrame(attn_rt)
attn_df

,probe_start,key press
0,64.5063,64.8923
1,69.1416,69.9268
2,74.1597,74.7441
3,78.9778,79.3309
4,83.6621,84.0474
5,88.2804,88.7985
6,93.0325,93.6668
7,97.8837,98.3351
8,102.5853,102.9694
9,107.1873,107.6377


In [49]:
log_file_rt = attn_df['key press'].astype('float64')-attn_df['probe_start'].astype('float64')
csv_rt = exp_raw[exp_raw['Trial Type']=='Presentation']['Attention Reaction Time (s)']
thisdict = {'log rt':log_file_rt, 'csv rt':csv_rt}
df = pd.DataFrame(thisdict)

In [50]:
df['diff'] = df['csv rt'] - df['log rt']
df

,log rt,csv rt,diff
0,0.3860,0.385950,-0.000050
1,0.7852,0.785256,0.000056
2,0.5844,0.584410,0.000010
3,0.3531,0.367908,0.014808
4,0.3853,0.385524,0.000224
5,0.5181,0.518086,-0.000014
6,0.6343,0.634352,0.000052
7,0.4514,0.451370,-0.000030
8,0.3841,0.384132,0.000032
9,0.4504,0.450264,-0.000136


In [51]:
print('Mean difference in response times between csv and log file: '+str(df['diff'].mean()))

Mean difference in response times between csv and log file: 0.0014951796722414423


In [52]:
print('Mean difference in response times between csv and log file, without outlier: '+str(df['diff'][[0,1,2,4,5,6,7,8,9]].mean()))

Mean difference in response times between csv and log file, without outlier: 1.5977308485250043e-05


### Responses from log file

In [53]:
list(log_file.loc[key_press_indices]['DATA'][3:])

['64.8923 \tDATA \tKeypress: 1',
 '69.9268 \tDATA \tKeypress: 3',
 '74.7441 \tDATA \tKeypress: 3',
 '79.3309 \tDATA \tKeypress: 1',
 '84.0474 \tDATA \tKeypress: 1',
 '88.7985 \tDATA \tKeypress: 3',
 '93.6668 \tDATA \tKeypress: 1',
 '98.3351 \tDATA \tKeypress: 1',
 '102.9694 \tDATA \tKeypress: 1',
 '107.6377 \tDATA \tKeypress: 1']

### Responses from csv

In [54]:
exp_raw[['Attention Button']].iloc[0:10]

,Attention Button
0,1.0
1,3.0
2,3.0
3,1.0
4,1.0
5,3.0
6,1.0
7,1.0
8,1.0
9,1.0
